In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
from send2trash import send2trash
from itertools import combinations
from itertools import product
from collections import defaultdict
import unicodedata
from datetime import datetime 
from webdriver_manager.chrome import ChromeDriverManager
import os
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC

os.chdir(r"/Users/jordandass/FPL")
teams = pd.read_csv('FDR.csv')
columns = list(teams["Team"])
os.chdir(r"/Users/jordandass/FPL/Transfer market")

def extractor(str_):
    dob = re.findall('[A-Za-z]{3,3} [0-9]{1,2}, [0-9]{4,4} \([0-9]{2,2}\)', str_)
    if dob == []:
        return [str_.strip()]
    else:
        dob = dob[0]
    val = re.sub('[A-Za-z]{3,3} [0-9]{1,2}, [0-9]{4,4} \([0-9]{2,2}\)', '', str_)
    if val == '':
        return [dob.strip()]
    else:
        return [dob.strip()] + [val.strip()]

driver = webdriver.Chrome(ChromeDriverManager().install())
team = columns[2]
driver.get("https://www.transfermarkt.com")
driver.find_element_by_xpath('//*[@id="schnellsuche"]/input[1]').send_keys(team)
driver.find_element_by_xpath('//*[@id="schnellsuche"]/input[1]').send_keys(Keys.RETURN)
time.sleep(5)
tables = driver.find_elements_by_class_name('items')
headers = driver.find_elements_by_class_name('table-header')

if len(tables) == 1:
    table = tables[0]
else:
    for table_, header in zip(tables, headers):
        if re.findall('SEARCH RESULTS: CLUBS', header.text):
            table = table_
            break

rows = table.find_elements_by_class_name('hauptlink')
for row in rows:
    print(row.text)
    time.sleep(2)
    row.click()
    
    num = 0
    lines = []
    while True:
        num += 1
        try:
            table_row = driver.find_element_by_xpath('//*[@id="yw1"]/table/tbody/tr['+ str(num) +']')
            lines.append(table_row.text)
        except:
            break
    array = []
    for line in lines:
        lists = []
        for element in line.split('\n'):
            lists += extractor(element)
        lists.append(team)
        array.append(lists)
    break


for team in columns[1:]:
    driver.find_element_by_xpath('//*[@id="schnellsuche"]/input[1]').send_keys(team)
    driver.find_element_by_xpath('//*[@id="schnellsuche"]/input[1]').send_keys(Keys.RETURN)
    time.sleep(5)
    tables = driver.find_elements_by_class_name('items')
    headers = driver.find_elements_by_class_name('table-header')
    
    if len(tables) == 1:
        table = tables[0]
    else:
        for table_, header in zip(tables, headers):
            if re.findall('SEARCH RESULTS: CLUBS', header.text):
                table = table_
                break
        
    rows = table.find_elements_by_class_name('hauptlink')
    for row in rows:
        print(row.text)
        time.sleep(2)
        row.click()
        num = 0
        lines = []
        while True:
            num += 1
            try:
                table_row = driver.find_element_by_xpath('//*[@id="yw1"]/table/tbody/tr['+ str(num) +']')
                lines.append(table_row.text)
            except:
                break
        for line in lines:
            lists = []
            for element in line.split('\n'):
                lists += extractor(element)
            lists.append(team)
            array.append(lists)
        break
display(pd.DataFrame(array, columns = ['No.', 'Player', 'Pos', 'DoB', 'Val', 'Team']))
pd.DataFrame(array, columns = ['No.', 'Player', 'Pos', 'DoB', 'Val', 'Team']).to_csv('MarketVal.csv')


[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/Users/jordandass/.wdm/drivers/chromedriver/mac64/95.0.4638.69/chromedriver] found in cache


Manchester City


ElementClickInterceptedException: Message: element click intercepted: Element <td class="hauptlink">...</td> is not clickable at point (195, 566). Other element would receive the click: <iframe src="https://cdn.privacy-mgmt.com/index.html?message_id=571579&amp;consentUUID=5ddd803f-ada4-4633-9fbd-b069ac5330d3&amp;requestUUID=ad99b66c-1dd2-4a43-a3d7-8ede54524cb7&amp;preload_message=true" id="sp_message_iframe_571579" title="SP Consent Message"></iframe>
  (Session info: chrome=95.0.4638.69)


In [50]:
import pandas as pd
import re

df = pd.read_csv('MarketVal.csv')
lists = []
for val in list(df['Val']):
    if re.findall('m', str(val)):
        num_val = re.sub('€|m', '', str(val))
        lists.append(float(num_val)*1000000)
    else:
        num_val = re.sub('€|Th.', '', str(val))
        lists.append(float(num_val)*1000)
df['CleanVal'] = lists
df.to_csv('MarketVal.csv')
    